In [ ]:
import numpy as np 
from qTools import *

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import datetime

# Define Input Parameters

In [ ]:
wq = 1
wc = 1
dim = 2
g = 1

In [ ]:
system = Qubit(frequency=wq) + Cavity(dimension=dim, frequency=wc)

coupling = system.JC(g)

system.initialState = identity(system.dimension)

In [ ]:
halfTime = 5
system.simulation.totalTime = 2*halfTime
system.simulation.stepCount = 150
system.simulation.delStates = True

In [ ]:
Amax = 1.5
Amin = 0.5
Bmax = 2.0
Bmin = 0.5
NA = 60 #Steps in Amplitude array
NB = 60 #Steps in Standard deviation array

AA = np.linspace(Amin, Amax, NA)
BB = np.linspace(Bmin, Bmax, NB)

ampSweep = system.simulation.Sweep.createSweep(system='Qubit1', sweepKey='amp', sweepList = AA)
bSweep = system.simulation.Sweep.createSweep(system='Qubit1', sweepKey='std',  sweepList = BB, multiParam=True)

timeDependency = system.simulation.timeDependency.createSweep(system='Qubit1')

def specialSweep(sweep):
    #compSys = sweep.getObjByName('QuantumSystem1')

    time = sweep.system.simulation.stepSize*sweep.index
    amplitude = sweep.system.aux['amp']
    std = sweep.system.aux['std']

    sweep.system.frequency = amplitude*np.exp(-(time-halfTime)**2/std**2)

timeDependency.sweepFunction = specialSweep

In [ ]:
def compute(qSim, *args):
    qSim.qRes.result = ('qubit freq', qSim.getObjByName('Qubit1').frequency)

def calculate(sys):
    res = sys.qRes
    unitary = sys.unitary

    res.result = ('00real', np.real(unitary[0,0]))
    res.result = ('11real', np.real(unitary[1,1]))
    res.result = ('22real', np.real(unitary[2,2]))
    res.result = ('33real', np.real(unitary[3,3]))

    res.result = ('00imag', np.imag(unitary[0,0]))
    res.result = ('11imag', np.imag(unitary[1,1]))
    res.result = ('22imag', np.imag(unitary[2,2]))
    res.result = ('33imag', np.imag(unitary[3,3]))

    res.result = ('03real', np.real(unitary[0,3]))
    res.result = ('30real', np.real(unitary[3,0]))

    res.result = ('03imag', np.imag(unitary[0,3]))
    res.result = ('30imag', np.imag(unitary[3,0]))

system.calculate = (calculate, 'end')
system.simulation.compute = compute

nw = datetime.datetime.now()

system.simulation.run(p=True, coreCount='all')

en = datetime.datetime.now()
print(en-nw)

In [ ]:
time = [system.simulation.stepSize*x for x in range(system.simulation.stepCount-1)]
plt.plot(time, system.simulation.qRes.results['qubit freq'][0][0])
plt.plot(time, system.simulation.qRes.results['qubit freq'][29][0])
plt.plot(time, system.simulation.qRes.results['qubit freq'][59][0])
plt.plot(time, system.simulation.qRes.results['qubit freq'][0][29])
plt.plot(time, system.simulation.qRes.results['qubit freq'][0][29])
plt.plot(time, system.simulation.qRes.results['qubit freq'][15][15])

In [ ]:
def pltcolor(axis, X, Y, Z):
    surf = axis.pcolormesh(Y,X,Z,  cmap='plasma') #vmin =-1, vmax=1
    divider = make_axes_locatable(axis)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(surf, cax=cax, orientation='vertical')
    axis.set_xlabel('Gauss std "B" variation (time)')
    axis.set_ylabel('Amp "A" Variation (Frequency)')


Y, X = np.meshgrid(BB,AA)


fig, ((ax1,ax7), (ax2,ax8), (ax3,ax9), (ax4,ax10), (ax5,ax11), (ax6,ax12)) = plt.subplots(nrows=6, ncols=2, figsize=(12,15))
pltcolor(ax1,X,Y, system.qRes.results['00real'])
ax1.set_title('Real Element [0,0]')
pltcolor(ax2,X,Y, system.qRes.results['11real'])
ax2.set_title('Real Element [1,1]')
pltcolor(ax3,X,Y, system.qRes.results['22real'])
ax3.set_title('Real Element [2,2]')
pltcolor(ax4,X,Y, system.qRes.results['33real'])
ax4.set_title('Real Element [3,3]')
pltcolor(ax5,X,Y, system.qRes.results['03real'])
ax5.set_title('Real Element [0,3]')
pltcolor(ax6,X,Y, system.qRes.results['30real'])
ax6.set_title('Real Element [3,0]')
pltcolor(ax7,X,Y, system.qRes.results['00imag'])
ax7.set_title('Imaginary Element [0,0]')
pltcolor(ax8,X,Y, system.qRes.results['11imag'])
ax8.set_title('Imaginary Element [1,1]')
pltcolor(ax9,X,Y, system.qRes.results['22imag'])
ax9.set_title('Imaginary Element [2,2]')
pltcolor(ax10,X,Y, system.qRes.results['33imag'])
ax10.set_title('Imaginary Element [3,3]')
pltcolor(ax11,X,Y, system.qRes.results['03imag'])
ax11.set_title('Imaginary Element [0,3]')
pltcolor(ax12,X,Y, system.qRes.results['30imag'])
ax12.set_title('Imaginary Element [3,0]')

plt.tight_layout()